In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import RobustScaler
from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn import metrics
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer

: 

In [10]:
class ModifiedSimpleImputer(SimpleImputer):
    def transform(self, X):
        return super().transform(X).flatten()

def getRegressResults(regress, parameters, data):

  # PREPROCESSING
  # Target
  y = data['averageRating'].dropna()

  # Features preprocessing
  X = data.drop('averageRating', axis=1)
  transfo_name = Pipeline(steps=[
        ('imputation', SimpleImputer(strategy='most_frequent')),
        ('transf', TargetEncoder()),
  ])

  transfo_num = Pipeline(steps=[
      ('imputation', SimpleImputer(strategy='median')),
      ('scaling', RobustScaler())
  ])
  
  transfo_genres = Pipeline(steps=[
      ('imputation', ModifiedSimpleImputer(strategy='most_frequent')),
      ('vec', CountVectorizer())
  ])

  preparation = ColumnTransformer(
      transformers=[
          ('data_num', transfo_num , X.select_dtypes(exclude=['object']).columns),
          ('genres', transfo_genres, ['genres']),
          ('names', transfo_name, ['directors_name', 'actors', 'actress'])
          
      ])

  # train-test-split	
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2, stratify=y)	

  # Pipeline and Model
  model = Pipeline(steps=[('preparation', preparation),
                          ('model', regress)])

  # Gridsearch
  grid = GridSearchCV(estimator = model, param_grid = parameters, scoring = 'r2', cv = 5, n_jobs =-1, verbose = 0)
  #grid = GridSearchCV(estimator = model, param_grid = parameters, scoring = 'r2', cv = 5, n_jobs =-1, verbose = 0)

  # Fit
  grid.fit(X_train, y_train)

  # Predict
  y_pred = grid.predict(X_test)
  test_score = metrics.r2_score(y_test, y_pred)

  # Results
  regress_results = []
  regress_results.append(grid.cv_results_['mean_fit_time'].mean().round(4))
  regress_results.append(grid.best_score_.round(4))
  regress_results.append(test_score.round(4))
  regress_results.append(grid.best_params_)

  return(regress_results)

def getRandomRegressResults(regress, parameters, data):

  # PREPROCESSING
  # Target
  y = data['averageRating'].dropna()

  # Features preprocessing
  X = data.drop('averageRating', axis=1)
  transfo_name = Pipeline(steps=[
        ('imputation', SimpleImputer(strategy='most_frequent')),
        ('transf', TargetEncoder()),
  ])

  transfo_num = Pipeline(steps=[
      ('imputation', SimpleImputer(strategy='median')),
      ('scaling', RobustScaler())
  ])
  
  transfo_genres = Pipeline(steps=[
      ('imputation', ModifiedSimpleImputer(strategy='most_frequent')),
      ('vec', CountVectorizer())
  ])

  preparation = ColumnTransformer(
      transformers=[
          ('data_num', transfo_num , X.select_dtypes(exclude=['object']).columns),
          ('genres', transfo_genres, ['genres']),
          ('names', transfo_name, ['directors_name', 'actors', 'actress'])
          
      ])

  # train-test-split	
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2, stratify=y)	

  # Pipeline and Model
  model = Pipeline(steps=[('preparation', preparation),
                          ('model', regress)])

  # Gridsearch
  random = RandomizedSearchCV(estimator = model, param_distributions= parameters, scoring = 'r2', n_jobs =-1, cv = 5, verbose = 0)
  #grid = GridSearchCV(estimator = model, param_grid = parameters, scoring = 'r2', cv = 5, n_jobs =-1, verbose = 0)

  # Fit
  random.fit(X_train, y_train)

  # Predict
  y_pred = random.predict(X_test)
  test_score = metrics.r2_score(y_test, y_pred)

  # Results
  regress_results = []
  regress_results.append(random.cv_results_['mean_fit_time'].mean().round(4))
  regress_results.append(random.best_score_.round(4))
  regress_results.append(test_score.round(4))
  regress_results.append(random.best_params_)

  return(regress_results)

In [11]:
df = pd.read_csv('datasets/data_regression.csv', index_col=0)
df = df.drop('tconst', axis=1)
df['genres'] = df['genres'].str.replace(',', ' ')
df

,averageRating,decade,runtimeMinutes,genres,directors_name,actors,actress
0,6.2,191.0,45.0,NaN,Urban Gad,Hans Mierendorff,Agda Nilsson
1,6.1,191.0,48.0,Comedy,Ernst Lubitsch,Harry Liedtke,Agda Nilsson
2,7.0,193.0,NaN,Crime,Rolf Randolf,Hans Mierendorff,Zet Molas
3,6.2,192.0,NaN,Action Adventure Drama,Joe May,Hans Mierendorff,Mia May
4,5.2,192.0,NaN,Drama,Joe May,Emil Jannings,Mia May
...,...,...,...,...,...,...,...
211617,6.2,202.0,101.0,Drama,Morgan Ingari,Michael Judson Berry,Molly Bernard
211618,6.3,201.0,91.0,Drama War,José Luis Rugeles,Carlos Clavijo,Carmenza González
211619,4.0,196.0,83.0,Horror,Joseph Adler,Ross Harris,Eugenie Wingate
211620,7.7,201.0,91.0,Thriller,Jon-Claire Lee,Asif Khan,Lucy Quill


In [268]:
model = GradientBoostingRegressor(n_estimators=500, max_depth=4)
parameters = {}

In [269]:
getRegressResults(model, parameters, data=df)

[173.5435, 0.1754, 0.1921, {}]

In [252]:
model = AdaBoostRegressor(n_estimators=175)
parameters = {}

In [253]:
getRegressResults(model, parameters, data=df)

[26.217, 0.1059, 0.0786, {'model__n_estimators': 175}]

In [4]:
model = RandomForestRegressor()
parameters = {}

In [9]:
getRegressResults(model, parameters, data=df)

[2656.06, 0.1549, 0.1687, {}]